<a href="https://colab.research.google.com/github/onuralpArsln/MlAiTutorialProjects/blob/main/14-WordEmbedings/Embedings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Burada RAG için önemli olacak bir yapıyı inceliyoruz text içinden konu ile alakalı şeyleri bulmak için bir text yöentim sistemi

Yazı → Embedding → Cosine Similarity → En alakalı içerikler → LLM'e ver → Süper tahmin


# Pre-Req


In [1]:
%pip install -q sentence-transformers scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.0 MB/s eta 0:00:00


# En Basit Hafıza Modelimiz

memory = Hem metin hem embedding saklıyoruz.

model = Ücretsiz, hafif bir encoder.

MAX_MEMORY = Hafızamız dolunca eskileri atıyoruz.

In [2]:
# 1. Gerekli modülleri alalım
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 2. Embedding modelimizi yükleyelim (çok hafif bir model)
model = SentenceTransformer('all-MiniLM-L6-v2')

# 3. Haber hafızamızı tutacak yer
memory = []

# 4. Sliding window kapasitesi
MAX_MEMORY = 5  # son 5 haber tutulacak

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Haberi / Yeni Metini Kaydetme Fonksiyonu

encode([text])[0] ➔ metni vektör yapıyor.

memory dizisine hem metin hem embedding ekliyoruz.

5'ten fazla olunca en eskisi çıkıyor ➔ SLIDING window böyle.

In [4]:
def add_news_to_memory(text):
    # Haber embed ediliyor (sayılara çeviriliyor)
    embedding = model.encode([text])[0]

    # Memory'ye kaydediyoruz
    memory.append({
        "text": text,
        "embedding": embedding
    })

    # Sliding window: Hafıza dolarsa en eskiyi sil
    if len(memory) > MAX_MEMORY:
        memory.pop(0)


# En Benzer Haberi / Metini Bulma Fonksiyonu

cosine_similarity ➔ 0 ile 1 arasında benzerlik verir.

1 ➔ aynı haber, 0 ➔ tamamen alakasız.

In [5]:
def find_most_similar_news(new_text):
    if not memory:
        return None

    # Yeni gelen haberi embed edelim
    new_embedding = model.encode([new_text])[0]

    # Hafızadaki embeddinglerle benzerlik hesapla
    memory_embeddings = np.array([item['embedding'] for item in memory])
    similarities = cosine_similarity([new_embedding], memory_embeddings)[0]

    # En yüksek skorlu haberi bul
    most_similar_idx = np.argmax(similarities)
    most_similar_text = memory[most_similar_idx]['text']
    similarity_score = similarities[most_similar_idx]

    return most_similar_text, similarity_score

# Kendi Kendimize Test Edelim!

In [11]:
# Hafızaya 5 haber atalım
add_news_to_memory("Borsa bugün %2 arttı, yatırımcılar mutlu.")
add_news_to_memory("Merkez Bankası faiz oranlarını değiştirmedi.")
add_news_to_memory("Dolar kuru sabit kaldı, euroda küçük düşüş.")
add_news_to_memory("Altın fiyatları rekor kırdı, yatırımcılar ilgilendi.")
add_news_to_memory("Teknoloji şirketlerinde büyük işten çıkarmalar başladı.")

# Yeni haber yazalım
new_news = "Borsa rekor seviyeye ulaştı, yatırımcılar coşkulu."

# En benzer eski haberi bulalım
similar_text, score = find_most_similar_news(new_news)

print(f"\nYeni haber: {new_news}\n")
print(f"En benzer eski haber: {similar_text}")
print(f"Benzerlik skoru: {score:.2f}")


Yeni haber: Borsa rekor seviyeye ulaştı, yatırımcılar coşkulu.

En benzer eski haber: Altın fiyatları rekor kırdı, yatırımcılar ilgilendi.
Benzerlik skoru: 0.83


# En Benzer 3 Haber Bulma

np.argsort(similarities)[::-1]: Benzerlikleri azalan sırayla sıraladık.

Sonra, ilk 3 elemana (yani en yüksek skorlara) erişiyoruz.

top_3_texts: En benzer 3 haberin metinlerini alıyoruz.

top_3_scores: En benzer 3 haberin skorlarını alıyoruz.

In [12]:
def find_top_3_similar_news(new_text):
    if not memory:
        return None

    # Yeni gelen haberi embed edelim
    new_embedding = model.encode([new_text])[0]

    # Hafızadaki embeddinglerle benzerlik hesapla
    memory_embeddings = np.array([item['embedding'] for item in memory])
    similarities = cosine_similarity([new_embedding], memory_embeddings)[0]

    # Benzerlikleri azalan sırayla sıralayalım (en yüksek skoru en önce almak için)
    sorted_idx = np.argsort(similarities)[::-1]

    # En benzer 3 haberi ve skorları al
    top_3_texts = []
    top_3_scores = []
    for idx in sorted_idx[:3]:
        top_3_texts.append(memory[idx]['text'])
        top_3_scores.append(similarities[idx])

    return top_3_texts, top_3_scores


In [13]:
# Yeni bir haber geldi
new_news = "Borsa rekor seviyeye ulaştı, yatırımcılar coşkulu."

# En benzer 3 haberi ve skorlarını bulalım
top_3_texts, top_3_scores = find_top_3_similar_news(new_news)

# Sonuçları yazdıralım
print("En benzer 3 haber ve skorları:")
for text, score in zip(top_3_texts, top_3_scores):
    print(f"- {text} (Skor: {score:.2f})")


En benzer 3 haber ve skorları:
- Altın fiyatları rekor kırdı, yatırımcılar ilgilendi. (Skor: 0.83)
- Borsa bugün %2 arttı, yatırımcılar mutlu. (Skor: 0.81)
- Dolar kuru sabit kaldı, euroda küçük düşüş. (Skor: 0.74)


# Embed Görselleştirmesi

In [ ]:
# 1. Gerekli modülleri yükle
%pip install -q matplotlib scikit-learn

In [ ]:


# 2. Kodlara başla
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np

# Hafızamızda birkaç haber olsun (örnek verelim)
memory = [
    {"text": "Borsa yükseldi, yatırımcılar mutlu.", "embedding": model.encode(["Borsa yükseldi, yatırımcılar mutlu."])[0]},
    {"text": "Merkez Bankası faiz oranlarını değiştirmedi.", "embedding": model.encode(["Merkez Bankası faiz oranlarını değiştirmedi."])[0]},
    {"text": "Dolar kuru sabit kaldı.", "embedding": model.encode(["Dolar kuru sabit kaldı."])[0]},
    {"text": "Altın fiyatları rekor kırdı.", "embedding": model.encode(["Altın fiyatları rekor kırdı."])[0]},
    {"text": "Teknoloji şirketlerinde işten çıkarmalar başladı.", "embedding": model.encode(["Teknoloji şirketlerinde işten çıkarmalar başladı."])[0]},
]

# 3. Embeddingleri toplayalım
embeddings = np.array([item['embedding'] for item in memory])

# 4. PCA ile 2D'ye indirelim
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embeddings)

# 5. Grafiği çizelim
plt.figure(figsize=(8,6))
for idx, item in enumerate(memory):
    x, y = reduced_embeddings[idx]
    plt.scatter(x, y)
    plt.text(x+0.01, y+0.01, item['text'], fontsize=9)

plt.title('Haber Embeddingleri PCA ile Görselleştirme')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.grid(True)
plt.show()
